In [2]:
import pandas as pd
from scipy.io import loadmat
from datautil import fault2RUL0, RULData_reduction

In [13]:
Rect_ripple=['i1','i2','i3','Load volt','Ripple','Vpp','R','L','C01','C02','C03','C1_f','C2_f','C3_f','fault']#
Var={'Rectifier':Rect_ripple}

## Rectifier

In [14]:
data='Rectifier' 
Test='' #'_Test' #
sim_name=f'{data}{Test}0.01' #
dataset= 'Rectifier' 

# Load Data

In [16]:
train_data=loadmat(f'./{data}/{sim_name}.mat')
data_name=list(train_data.keys())[3]
N=train_data[data_name][0].shape[0] #numberN
print(f'number of systems: {N}')

number of systems: 200


# Create PandasDataset

In [17]:
variables=Var[data]#Rect_DiodeVar # 

data_df=pd.DataFrame()
for sys in range(N):
    rect_data=train_data[data_name][0][sys]
    df=pd.DataFrame(rect_data,columns=variables)
    df['System']=[(sys+1)]*rect_data.shape[0]
    data_df=pd.concat([data_df, df], axis=0)  

In [18]:
pd.options.mode.chained_assignment = None  #for false positive warning

if dataset == 'Rectifier':
    Count_df=data_df[[f'C{i+1}_f' for i in range(3)]+['fault']+['System']] 
    Count_df['Sum']=Count_df[[f'C{i+1}_f' for i in range(3)]+['fault']].sum(axis=1)
    s=Count_df.groupby('System')['Sum'].last()
    for i in range(5):
        print(f' {i} capacitors(include System) failed {(s==i).sum()} times')

 0 capacitors(include System) failed 0 times
 1 capacitors(include System) failed 0 times
 2 capacitors(include System) failed 0 times
 3 capacitors(include System) failed 9 times
 4 capacitors(include System) failed 191 times


# Convert Fault signals to RUL

In [19]:
df=fault2RUL0(data_df,system='System',fault='fault')#Ar_train
if dataset in ['RectifierRUL','Rectifier']:
    for i in range(3): #3
        df=fault2RUL0(df,system='System',fault=f'C{i+1}_f')

In [20]:
n=len(df.loc[df["fault"]==1])
print(f'Number of faulty systems {n}/{N}')

Number of faulty systems 200/200


## Create System dataset

In [10]:
Sys_ext_df=df[['System','i1','i2','i3','Load volt','R','L','C01','C02','C03','fault']]#'Vpp','Load volt',

Sys_ext_df.rename(columns={'fault':'RUL'}, inplace=True);


In [11]:
Sys_ext_df

,System,i1,i2,i3,Load volt,R,L,C01,C02,C03,RUL
0,1,-0.001269,0.000421,0.000912,-0.000945,1.078040,0.108574,0.001997,0.001868,0.001898,338.0
1,1,1.361276,1.275093,1.294115,3.983146,1.078040,0.108574,0.001997,0.001868,0.001898,337.0
2,1,0.311968,0.293476,0.294862,4.652862,1.078040,0.108574,0.001997,0.001868,0.001898,336.0
3,1,0.391343,0.365750,0.371482,4.439287,1.078040,0.108574,0.001997,0.001868,0.001898,335.0
4,1,0.498769,0.467268,0.473573,4.185995,1.078040,0.108574,0.001997,0.001868,0.001898,334.0
...,...,...,...,...,...,...,...,...,...,...,...
396,200,1.009320,1.043419,0.984695,2.932096,0.925813,0.113398,0.002107,0.002126,0.002101,0.0
397,200,1.009768,1.045552,0.986249,2.932405,0.925813,0.113398,0.002107,0.002126,0.002101,0.0
398,200,1.010723,1.043725,0.984879,2.931965,0.925813,0.113398,0.002107,0.002126,0.002101,0.0
399,200,1.011373,1.044565,0.983559,2.932863,0.925813,0.113398,0.002107,0.002126,0.002101,0.0


In [12]:
pd.set_option('display.max_rows', 100)
life=Sys_ext_df.groupby('System').first()
life

,i1,i2,i3,Load volt,R,L,C01,C02,C03,RUL
System,,,,,,,,,,
1,-0.001269,0.000421,0.000912,-0.000945,1.078040,0.108574,0.001997,0.001868,0.001898,338.0
2,0.000096,-0.000787,-0.000055,-0.002729,0.940276,0.114835,0.001970,0.002280,0.001972,295.0
3,-0.000317,0.002654,-0.001465,-0.000080,1.051327,0.116126,0.001967,0.002035,0.001801,318.0
4,0.000243,0.000477,-0.001305,0.000307,1.136495,0.091045,0.002213,0.001869,0.002019,354.0
5,-0.001597,-0.000623,0.001471,-0.000545,1.125643,0.108439,0.001851,0.001941,0.002254,322.0
...,...,...,...,...,...,...,...,...,...,...
196,-0.001967,0.001936,-0.001094,0.001964,0.903153,0.091073,0.002225,0.002171,0.002131,340.0
197,-0.000347,0.001229,0.000298,-0.001360,1.048382,0.091363,0.002109,0.002285,0.002135,299.0
198,0.000477,-0.000208,-0.000089,0.001247,1.053202,0.107144,0.002035,0.002137,0.002179,319.0


## Save data

In [12]:
Sys_ext_df.to_csv(f'./{data}/{dataset}{Test}_ext.csv',index=False)
Sys_df=RULData_reduction(Sys_ext_df)
Sys_df.to_csv(f'./{data}/{dataset}{Test}.csv',index=False)

In [13]:
Sys_df

,System,i1,i2,i3,Load volt,R,L,C01,C02,C03,RUL
0,1,-0.001269,0.000421,0.000912,-0.000945,1.078040,0.108574,0.001997,0.001868,0.001898,338.0
1,1,1.361276,1.275093,1.294115,3.983146,1.078040,0.108574,0.001997,0.001868,0.001898,337.0
2,1,0.311968,0.293476,0.294862,4.652862,1.078040,0.108574,0.001997,0.001868,0.001898,336.0
3,1,0.391343,0.365750,0.371482,4.439287,1.078040,0.108574,0.001997,0.001868,0.001898,335.0
4,1,0.498769,0.467268,0.473573,4.185995,1.078040,0.108574,0.001997,0.001868,0.001898,334.0
...,...,...,...,...,...,...,...,...,...,...,...
338,200,1.013128,1.045791,0.990074,2.932182,0.925813,0.113398,0.002107,0.002126,0.002101,4.0
339,200,1.014701,1.044371,0.990318,2.931330,0.925813,0.113398,0.002107,0.002126,0.002101,3.0
340,200,1.014021,1.046734,0.991708,2.933881,0.925813,0.113398,0.002107,0.002126,0.002101,2.0
341,200,1.012124,1.045213,0.989775,2.929885,0.925813,0.113398,0.002107,0.002126,0.002101,1.0


In [14]:
Sys_df.groupby('System').nth(0)

,System,i1,i2,i3,Load volt,R,L,C01,C02,C03,RUL
0,1,-0.001269,0.000421,0.000912,-0.000945,1.078040,0.108574,0.001997,0.001868,0.001898,338.0
0,2,0.000096,-0.000787,-0.000055,-0.002729,0.940276,0.114835,0.001970,0.002280,0.001972,295.0
0,3,-0.000317,0.002654,-0.001465,-0.000080,1.051327,0.116126,0.001967,0.002035,0.001801,318.0
0,4,0.000243,0.000477,-0.001305,0.000307,1.136495,0.091045,0.002213,0.001869,0.002019,354.0
0,5,-0.001597,-0.000623,0.001471,-0.000545,1.125643,0.108439,0.001851,0.001941,0.002254,322.0
...,...,...,...,...,...,...,...,...,...,...,...
0,196,-0.001967,0.001936,-0.001094,0.001964,0.903153,0.091073,0.002225,0.002171,0.002131,340.0
0,197,-0.000347,0.001229,0.000298,-0.001360,1.048382,0.091363,0.002109,0.002285,0.002135,299.0
0,198,0.000477,-0.000208,-0.000089,0.001247,1.053202,0.107144,0.002035,0.002137,0.002179,319.0
0,199,0.000387,-0.001880,-0.000096,0.000573,1.074530,0.086272,0.002279,0.002012,0.002080,308.0
